In [1]:
from bula_process.models import Mistral, Rag, Llama3
import json 

model = Llama3()
rag = Rag()

system_prompt = """
Always answer the question in Brazilian Portuguese, being as simple and concise as possible.
Return ONLY the answer.
Example:
Quais efeitos colaterais podem ocorrer com a administração prolongada do Mud Oral?
Answer:
Hipertensão arterial e taquicardia.
"""

llm_judge_prompt = """
Given this question: {question}, this correct answer: {right_answer} and this wrong answer: {wrong_answer}, judge if this generated answer is correct or wrong: {generated_answer}
Correct means the answer answers the question. Wrong means the opposite.
Return ONLY "Correct" if the answer is correct, and "Wrong" if the answer is wrong. 
"""

llm_judge_prompt_2 = """
Considering the question '{question}' and the following answer '{generated_answer}', a similar alternative answer is: 
"""

llm_judge_prompt_2 = """
Considering the question '{question}' and the following answer '{generated_answer}', is '{option}' a similar alternative answer?
"""

llm_judge_prompt_3 = """
Considering the question '{question}' and the following answer '{generated_answer}', select the option with the most similar alternative answer: 
{options}
e) None of the options is similar to the text.
Option:
"""

llm_judge_perplexity_prompt = """
Given this question: {question}, this correct answer: {right_answer} and this wrong answer: {wrong_answer}, judge if this generated answer is correct or wrong: {generated_answer}
Correct means the answer answers the question. Wrong means the opposite.
Return ONLY "Correct" if the answer is correct, and "Wrong" if the answer is wrong. 
The generated answer is:  
"""

# llm_judge_explain_prompt = """
# Given this question: {question}, this correct answer: {right_answer} and this wrong answer: {wrong_answer}, judge if this generated answer is correct or wrong: {generated_answer}
# Explain why is correct and why is wrong. 
# """


path = 'results/bula_all.jsonl'
list_json = []
with open(path, 'r') as file:
    for line in file:
        list_json.append(json.loads(line))

def make_question_rag(id, use_rag):
    data = list_json[id]
    question_prompt = ''
    if use_rag:
        context = rag.retrieve(data['query'])
        question_prompt = f"[INST]Answer the question: [/INST] Context: {context} \n {data['query']} \nAnswer: "
    else:
        question_prompt = f"[INST]Answer the question: [/INST] \n {data['query']} \nAnswer: "
   
    answer_model = model.inference(system_prompt + question_prompt)   

    gold_choice = ''
    wrong_choice = ''
    for idx, choice in enumerate(data['choices']):
        if idx == int(data['gold']):
            gold_choice = choice
        else:
            wrong_choice = choice
        perplexity_question = model.perplexity(llm_judge_prompt_2.format(question=data['query'], right_answer=gold_choice, wrong_answer=wrong_choice, generated_answer=answer_model))
        
    judge_veredict = model.inference(llm_judge_prompt.format(question=data['query'], right_answer=gold_choice, wrong_answer=wrong_choice, generated_answer=answer_model))

    perplexity_right = model.perplexity(llm_judge_perplexity_prompt.format(question=data['query'], right_answer=gold_choice, wrong_answer=wrong_choice, generated_answer=answer_model), "Correct.")
    perplexity_wrong = model.perplexity(llm_judge_perplexity_prompt.format(question=data['query'], right_answer=gold_choice, wrong_answer=wrong_choice, generated_answer=answer_model), "Wrong.")

    print(data['query'])
    print(f"Resposta correta: {gold_choice}")
    print(f"Resposta gerada: {answer_model}")
    print(f"LLM Judge as: {judge_veredict}")
    print(f"Perplexity Right: {perplexity_right:.2f}")
    print(f"Perplexity Wrong: {perplexity_wrong:.2f}")



/home/navarro/miniconda3/envs/python310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.71it/s]
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub

In [2]:
make_question_rag(109, True)

Quais são os sintomas de efeitos colaterais que podem ocorrer com o uso de Neo Mistatin creme?
Resposta correta: irritação e sensibilidade, incluindo sensação de queimação e coceira
Resposta gerada: Irritação, sensibilidade, queimação e prurido.
LLM Judge as: Correct
Perplexity Right: 13.14
Perplexity Wrong: 167.97


In [3]:
make_question_rag(109, False)

Quais são os sintomas de efeitos colaterais que podem ocorrer com o uso de Neo Mistatin creme?
Resposta correta: irritação e sensibilidade, incluindo sensação de queimação e coceira
Resposta gerada: Erupção cutânea e prurido.
LLM Judge as: Correct
Perplexity Right: 42.60
Perplexity Wrong: 9.50


In [4]:
make_question_rag(95, True)

Qual é a reação muito comum (>1/10) que pode ocorrer com o uso de Oxaliplatina e é relacionada à função hematológica?
Resposta correta: Anemia
Resposta gerada: Leucopenia.
LLM Judge as: Correct
Perplexity Right: 19.41
Perplexity Wrong: 14.91


In [5]:
make_question_rag(95, False)

Qual é a reação muito comum (>1/10) que pode ocorrer com o uso de Oxaliplatina e é relacionada à função hematológica?
Resposta correta: Anemia
Resposta gerada: Anemia.
LLM Judge as: Correct
Perplexity Right: 3.73
Perplexity Wrong: 53.51


In [6]:
make_question_rag(3, True)

Quais são os efeitos colaterais comuns que podem ocorrer com o uso de Cefuroxima sódica?
Resposta correta: Diminuição das células de defesa do organismo (neutrófilos), aumento da quantidade de eosinófilos (células brancas) no sangue; aumento transitório das enzimas do fígado
Resposta gerada: Neutropenia, eosinofilia, aumento transitório das enzimas hepáticas, reações no local da injeção.
LLM Judge as: Correct
Perplexity Right: 8.38
Perplexity Wrong: 61.94


In [7]:
make_question_rag(3, False)

Quais são os efeitos colaterais comuns que podem ocorrer com o uso de Cefuroxima sódica?
Resposta correta: Diminuição das células de defesa do organismo (neutrófilos), aumento da quantidade de eosinófilos (células brancas) no sangue; aumento transitório das enzimas do fígado
Resposta gerada: Diarria, náusea e dor de cabeça.
LLM Judge as: Correct
Perplexity Right: 19.38
Perplexity Wrong: 12.86


In [8]:
make_question_rag(5, True)

Qual é o efeito colateral comum do PEYONA que ocorre entre 1% e 10% dos pacientes que utilizam este medicamento?
Resposta correta: Reações inflamatórias locais no lugar da perfusão
Resposta gerada: Flebite e alterações inflamatórias locais.
LLM Judge as: Correct
Perplexity Right: 17.08
Perplexity Wrong: 11.38


In [9]:
make_question_rag(5, False)

Qual é o efeito colateral comum do PEYONA que ocorre entre 1% e 10% dos pacientes que utilizam este medicamento?
Resposta correta: Reações inflamatórias locais no lugar da perfusão
Resposta gerada: Dor de cabeça.
LLM Judge as: Wrong
Perplexity Right: 58.22
Perplexity Wrong: 11.45


In [10]:
make_question_rag(8, True)

Quais são os efeitos colaterais mais comuns que podem ocorrer com o uso de Hidroquinona?
Resposta correta: Vermelhidão, coceira, descamação, inflamação excessiva, vesículas (bolhas) e sensação leve de queimação
Resposta gerada: Eritemia, sensação leve de queimação e hipersensibilidade ocasional.
LLM Judge as: Correct
Perplexity Right: 14.88
Perplexity Wrong: 65.11
